# Identify and Download Datasets

### Purpose

The purpose of this notebook is to identify and download the datasets with pH and pCO2 data deployed at the Ocean Observatory Initiative's (OOI) Global Irminger Array (60$^{\circ}$N, 39$^{\circ}$W). OOI deploys the following sensors for measuring the ocean carbon system: Sunburst Sensors, LLC. SAMI-pH (PHSEN) pH, SAMI-pCO$_{2}$ seawater measurements, and the Pro-Oceanus pCO2 sensor measurements. The datasets identified here are used in the **```Data_Analysis```** notebook. 


### Datasets
While this notebook provides an example of interacting via Machine-to-Machine (M2M) with the OOI API in order to search for datasets, the tables below list the PHSEN, PCO2A, and PCO2W instruments deployed at the Global Irminger Array. Additionally, it identifies and lists associated CTD datasets that are needed for the relevant temperature, salinity, pressure, and density data.

#### PCO2A

| Instrument | Reference Designator | Mooring | Depth | Associated CTD Reference Designator |
| ---------- | -------------------- | ------- | ----- | ----------------------------------- |
| PCO2A      | GI01SUMO-SBD12-04-PCO2AA000 | Apex Surface Mooring | Surface | GI01SUMO-SBD12-06-METBKA000 |


#### PCO2W

| Instrument | Reference Designator | Mooring | Depth | Associated CTD Reference Designator |
| ---------- | -------------------- | ------- | ----- | ----------------------------------- |
| PCO2W      | GI01SUMO-RID16-05-PCO2WB000 | Apex Surface Mooring | 12 m | GI01SUMO-RID16-03-CTDBPF000 |
| PCO2W      | GI01SUMO-RII11-02-PCO2WC051 | Apex Surface Mooring | 40 m | GI01SUMO-RII11-02-CTDMOQ031 |
| PCO2W      | GI01SUMO-RII11-02-PCO2WC052 | Apex Surface Mooring | 80 m | GI01SUMO-RII11-02-CTDBPP032 |
| PCO2W      | GI01SUMO-RII11-02-PCO2WC053 | Apex Surface Mooring | 130 m | GI01SUMO-RII11-02-CTDBPP033 |

#### PHSEN
| Instrument | Reference Designator | Mooring | Depth | Associated CTD Reference Designator |
| ---------- | -------------------- | ------- | ----- | ----------------------------------- |
| PHSEN      | GI01SUMO-RII11-02-PHSENE041 | Apex Surface Mooring | 20 m | GI01SUMO-RII11-02-CTDMOQ011 |
| PHSEN      | GI01SUMO-RII11-02-PHSENE042 | Apex Surface Mooring | 100 m | GI01SUMO-RII11-02-CTDMOQ013 |
| PHSEN      | GI03FLMA-RIS01-04-PHSENF000 | Flanking Mooring A | 30 m | GI03FLMA-RIM01-02-CTDMOG040 |
| PHSEN      | GI01SUMO-RII11-02-CTDMOQ013 | Flanking Mooring B | 30 m | GI01SUMO-RII11-02-CTDMOQ013 |

### Methods

I use the following procedure for identifying the OOI carbon system and associated CTD datasets from OOI, requesting metadata, downloading the data, and preparing and cleaning the data. This notebook makes use of the **```OOINet```** module hosted at https://github.com/reedan88/OOINet as well as some functions from https://github.com/oceanobservatories/ooi-data-explorations. In order to access and download data from OOI via the API, you MUST register at https://ooinet.oceanobservatories.org and save your **username** and **api token** in a yaml (or similar) file.

First, the OOI API is pinged for available datasets as well as their common (or "English") names. Then, a specific reference designator (refdes) is selected. Then, I request the metadata for the given refdes. This returns a json file which is reshaped into a dataframe which contains all of the available data stream, data variables, data types, etc for the given reference designator. The metadata is used to filter for the relevant data streams to request from OOI Gold Copy THREDDS server. The Gold Copy THREDDS is utilized because it is a mostly static collection of OOI datasets which are updated only once-a-day, and is thus much faster to request and download. Note, however, that if you need very recent data you will have to use the base THREDDS server and that while most instruments are in the Gold Copy, there are few datasets which have not been migrated.

Next, with a selected "method" and "datastream" to request a dataset, we pass that to OOI API to get the download URL for the dataset. We then 

In [ ]:
import os, gc, sys
import json
import yaml
import numpy as np
import pandas as pd
import xarray as xr
import re
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Import OOI M2M tools
sys.path.append("/home/andrew/Documents/OOI-CGSN/ooinet/ooinet/")
from m2m import M2M

#### Set OOINet API access
In order access and download data from OOINet, need to have an OOINet api username and access token. Those can be found on your profile after logging in to OOINet. Your username and access token should NOT be stored in this notebook/python script (for security). It should be stored in a yaml file, kept in the same directory, named user_info.yaml.

In [ ]:
# Import user info for connecting to OOINet via M2M
userinfo = yaml.load(open("../../../../QAQC_Sandbox/user_info.yaml"), Loader=yaml.FullLoader)
username = userinfo["apiname"]
token = userinfo["apikey"]

#### Connect to OOINet

In [ ]:
OOINet = M2M(username, token)

---
## Datasets

Identify all of the OOI-CGSN datasets with the **```PCO2W```**, **```PCO2A```**, and **```PHSEN```** datasets that are located at the Global Irminger Array. This is done by querying OOINet and iteratively walking through all of the API endpoints. The results are saved into a csv file so this step doesn't have to be repeated each time.

Check to see if the reference designators have already been identified. If they haven't been previously downloaded, can use the ```OOINet.search_datasets``` function to search for the datasets associated with each instrument.

#### PCO2W

In [ ]:
try:
    # If the reference designators where previously identified and downloaded
    pco2w_datasets = pd.read_csv("../data/pco2w_datasets.csv")
except:
    # Search for PCO2W datasets, asking for English names
    pco2w_datasets = OOINet.search_datasets(instrument="PCO2W", English_names=True)

    # Save the datasets locally to speed up future runs
    pco2w_datasets.to_csv("../data/pco2w_datasets.csv", index=False)

# Print out the head
pco2w_datasets.head()

#### PHSEN

In [ ]:
try:
    # If the reference designators where previously identified and downloaded
    phsen_datasets = pd.read_csv("../data/phsen_datasets.csv")
except:
    # Search for PHSEN datasets, asking for full English names
    phsen_datasets = OOINet.search_datasets(instrument="PHSEN", English_names=True)

    # Save the datasets locally to speed up future runs
    phsen_datasets.to_csv("../data/phsen_datasets.csv", index=False)
    
phsen_datasets.head()

#### PCO2A

In [ ]:
try:
    # If the reference designators where previously identified and downloaded
    pco2a_datasets = pd.read_csv("../data/pco2a_datasets.csv")
except:
    # Search for PCO2A datasets
    pco2a_datasets = OOINet.search_datasets(instrument="PCO2A", English_names=True)

    # Save the datasets locally to speed up future runs
    pco2a_datasets.to_csv("../data/pco2a_datasets.csv", index=False)
    
pco2a_datasets.head()

Filter the datasets for the Irminger Array datasets, which start with the prefix "GI" for Global Irminger

In [ ]:
# PCO2A
mask = pco2a_datasets["array"].apply(lambda x: True if x.startswith("GI") else False)
pco2a_datasets = pco2a_datasets[mask]

# PCO2W
mask = pco2w_datasets["array"].apply(lambda x: True if x.startswith("GI") else False)
pco2w_datasets = pco2w_datasets[mask]

# PHSEN
mask = phsen_datasets["array"].apply(lambda x: True if x.startswith("GI") else False)
phsen_datasets = phsen_datasets[mask]

#### CTD & METBK
We will also need the temperature, salinity, and pressure data associated with the carbon system datasets. So we will also identify all the **```CTD```** datasets located at the Global Irminger Array as well as the **```METBK```** dataset for the surface mooring, which has the surface temperature and salinity.

In [ ]:
try:
    # If the reference designators where previously identified and downloaded
    ctd_datasets = pd.read_csv("../data/ctd_datasets.csv")
except:
    # Search for PCO2W datasets, asking for English names
    ctd_datasets = OOINet.search_datasets(instrument="CTD", English_names=True)

    # Save the datasets locally to speed up future runs
    ctd_datasets.to_csv("../data/ctd_datasets.csv", index=False)

# Print out the head
ctd_datasets.head()

Filter the datasets for the Irminger Array datasets, which start with the prefix "GI" for Global Irminger. For the **```CTD```** datasets, we also need to remove the Mobile Asset and Profiler datasets.

In [ ]:
# Identify the Global Irminger Array datasets
mask = ctd_datasets["array"].apply(lambda x: True if x.startswith("GP") else False)
ctd_datasets = ctd_datasets[mask]

# Remove datasets which are either Glider, AUV, or Profiler Mooring datasets
mask = ctd_datasets["refdes"].apply(lambda x: False if "MOAS" in x or "CTDPF" in x else True)
ctd_datasets = ctd_datasets[mask]

In [ ]:
try:
    # If the reference designators where previously identified and downloaded
    metbk_datasets = pd.read_csv("../data/metbk_datasets.csv")
except:
    # Search for PCO2W datasets, asking for English names
    metbk_datasets = OOINet.search_datasets(instrument="METBK", English_names=True)

    # Save the datasets locally to speed up future runs
    metbk_datasets.to_csv("../data/metbk_datasets.csv", index=False)

# Print out the head
metbk_datasets.head()

Filter the datasets for the Global Irminger dataset

In [ ]:
mask = metbk_datasets["array"].apply(lambda x: True if x.startswith("GI") else False)
metbk_datasets = metbk_datasets[mask]

---
## Download Datasets
Now, download the PCO2A, PHSEN, and PCO2W datasets along with their associated CTD datasets from OOINet and save locally for ease of access. We can scroll through the available datasets to identify which CTD datasets are 

### Irminger Array
* GI01SUMO: Apex Surface Mooring
    * SBD12: Surface Buoy
        * PCO2AA: pCO2 Air-Sea (refdes = GI01SUMO-SBD12-04-PCO2AA000)
        * METBKA: Bulk Meteorology Instrument Package (refdes = GI01SUMO-SBD12-06-METBKA000)
    * RID16: Near-Surface Instrument Frame
        * PCO2WB: pCO2 Water (refdes = GI01SUMO-RID16-05-PCO2WB000)
        * CTDBPF: CTD (refdes = GI01SUMO-RID16-03-CTDBPF000)
    * RII11: Mooring Riser
        * PCO2WC: pCO2 Water (40 meters) (refdes = GI01SUMO-RII11-02-PCO2WC051)
        * CTDMOQ: CTD (40 meters) (refdes = GI01SUMO-RII11-02-CTDBPP031)
        * PCO2WC: pCO2 Water (80 meters) (refdes = GI01SUMO-RII11-02-PCO2WC052)
        * CTDBPP: CTD (80 meters) (refdes = GI01SUMO-RII11-02-CTDBPP032)
        * PCO2WC: pCO2 Water (130 meters) (refdes = GI01SUMO-RII11-02-PCO2WC053)
        * CTDBPP: CTD (130 meters) (refdes = GI01SUMO-RII11-02-CTDBPP033)
        * PHSENE: Seawater pH (20 meters) (refdes = GI01SUMO-RII11-02-PHSENE041)
        * CTDMOQ: CTD (20 meters) (refdes = GI01SUMO-RII11-02-CTDMOQ011)
        * PHSENE: Seawater pH (100 meters) (refdes = GI01SUMO-RII11-02-PHSENE042)
        * CTDMOQ: CTD (100 meters) (refdes = GI01SUMO-RII11-02-CTDMOQ013)
* GI03FLMA: Flanking Subsurface Mooring A
    * RIS01: Mooring Riser
        * PHSENF: Seawater pH (refdes = GI03FLMA-RIS01-04-PHSENF000)
        * CTDMOG: CTD (30 meters) (refdes = GI03FLMA-RIM01-02-CTDMOG040)
* GI03FLMB: Flanking Subsurface Mooring B
    * RIS01: Mooring Riser
        * PHSENF: Seawater pH (refdes = GI03FLMB-RIS01-04-PHSENF000)
        * CTDMOG: CTD (30 meters) (refdes = GI03FLMB-RIM01-02-CTDMOG060)

### Surface Buoy: PCO2A (GI01SUMO-SBD12-04-PCO2AA000)

In [ ]:
refdes = "GP03FLMA-RIS01-04-PHSENF000"

In [ ]:
metadata = OOINet.get_metadata(refdes)
metadata = metadata.groupby(by=["refdes","method","stream"]).agg(lambda x: pd.unique(x.values.ravel()).tolist())
metadata = metadata.reset_index()
metadata = metadata.applymap(lambda x: x[0] if len(x) == 1 else x)
metadata

Occasionally, we need to further filter the available data streams from the metadata. For example, the METBK returns both computed flux products in the hourly datastream as well as a datastream with the measured met and sea surface data.

In [ ]:
datastreams = OOINet.get_datastreams(refdes)

# For METBK: Drop the hourly data streams
#mask = datastreams["stream"].apply(lambda x: True if "hourly" not in x else False)
#datastreams = datastreams[mask]
datastreams

#### Download the datasets

First, we define a function to try to eliminate "empty" datasets from the catalog. This occassionally happens depending on the instrument sampling schema, how it records data, and the requested time period.

In [ ]:
def clean_catalog(catalog, stream):
    """Clean up the netCDF catalog of unwanted datasets"""
    # Parse the netCDF datasets to only get those with the datastream in its name
    datasets = []
    for dset in catalog:
        check = dset.split("/")[-1]
        if stream in check:
            datasets.append(dset)
        else:
            pass
    
    # Next, check that the netCDF datasets are not empty by getting the timestamps in the
    # datasets and checking if they are 
    catalog = datasets
    datasets = []
    for dset in catalog:
        # Get the timestamps
        timestamps = dset.split("_")[-1].replace(".nc","").split("-")
        t1, t2 = timestamps
        # Check if the timestamps are equal
        if t1 == t2:
            pass
        else:
            datasets.append(dset)
            
    return datasets

Now, we iterate over each of the available data streams we identified with good data from the metadata, request the data, open the data into an xarray dataset, and save the dataset to a local directory.

In [ ]:
OOINet.REFDES = refdes
refdes

In [ ]:
for ind in metadata.index:
    row = metadata.loc[ind]
    method, stream = row["method"], row["stream"]
    if "power" in stream or "blank" in stream or "metadata" in stream or "control" in stream or "offset" in stream:
        continue
    else:
        pass
    
    # Get the thredds url
    thredds_url = OOINet.get_thredds_url(refdes, method, stream, goldCopy=True)
    print(thredds_url + "\n")
    # Access the catalog
    catalog = OOINet.get_thredds_catalog(thredds_url)
    # Parse the catalog for relevant netCDF files
    catalog = OOINet.parse_catalog(catalog, exclude=["gps", "blank"])
    catalog = sorted(catalog) 
    # Clean the catalog
    catalog = clean_catalog(catalog, stream)
       
    # Open the data
    data = OOINet.load_netCDF_datasets(catalog, goldCopy=True)
    
    # Eliminate unneeded timestamps
    for var in data.variables:
        if "time" in var and var != "time":
            data = data.drop_vars(var)
        elif data[var].dtype == "object":
            data = data.drop_vars(var)
        else:
            pass
            
    # Download and add annotations
    annotations = OOINet.get_annotations(refdes)
    data = OOINet.add_annotation_qc_flag(data, annotations)
    
    # Check that a suitable save directory exists
    saveDir = f"../data/{refdes}/"
    if not os.path.exists(saveDir):
        os.makedirs(saveDir)
    
    # Save the dataset
    filename = f"{refdes}-{method}-{stream}.nc"
    data.to_netcdf(saveDir+filename, engine="h5netcdf")

### Combine Datasets

Now, we need to merge the data. First, we iterate through the data variables for each dataset, identify any which are unique to a given dataset, and broadcast them to the other datasets. This step is necessary to allow the datasets to combine. Once each dataset has the same data variables, we utilize ```xr.combine_first``` to combine the datasets. We assume that the instrument record, if available, is the best and most complete dataset and utilize the telemetered and recovered_host datasets to fill in the gaps.

In [ ]:
sys.path.append("../../OS2022/OS2022/")

In [ ]:
import process_pco2w, process_phsen

In [ ]:
refdes = refdes
refdes

In [ ]:
saveDir = f"../data/{refdes}/"
for file in os.listdir(saveDir):
    if file.endswith(".nc"):
        if "recovered_inst" in file:
            inst_data = xr.open_dataset(saveDir + file, chunks="auto")
            if "PCO2W" in refdes:
                inst_data = process_pco2w.pco2w_instrument(inst_data)
                inst_data.to_netcdf(saveDir+filename, engine="h5netcdf")
            elif "PHSEN" in refdes:
                inst_data = process_phsen.phsen_instrument(inst_data)
            else:
                pass
        elif "recovered_host" in file:
            host_data = xr.open_dataset(saveDir + file, chunks="auto")
            if "PCO2W" in refdes:
                host_data = process_pco2w.pco2w_datalogger(host_data)
            elif "PHSEN" in refdes:
                host_data = process_phsen.phsen_datalogger(host_data)
            else:
                pass
        elif "telemetered" in file:
            tele_data = xr.open_dataset(saveDir + file, chunks="auto")
            if "PCO2W" in refdes:
                tele_data = process_pco2w.pco2w_datalogger(tele_data)
            elif "PHSEN" in refdes:
                tele_data = process_phsen.phsen_datalogger(tele_data)
            else:
                pass
        else:
            pass
        # Save the data
        

Open each method of data delivery and eliminate possible duplicate entries

In [ ]:
tele_data = tele_data.sel(time=~tele_data.get_index("time").duplicated())
host_data = host_data.sel(time=~host_data.get_index("time").duplicated())
inst_data = inst_data.sel(time=~inst_data.get_index("time").duplicated())

Next, need to make sure each dataset has the same variables in order to be able to combine. Can acheive this by iterating over the data variables for each method and utilizing ```xr.broadcast_like``` to create empty arrays for the datasets which are missing the variable.

In [ ]:
# Need to make sure each dataset has the same variables
for var in tele_data.variables:
    if var not in host_data.variables:
        host_data[var] = tele_data[var].broadcast_like(host_data["time"])
        
for var in host_data.variables:
    if var not in tele_data.variables:
        tele_data[var] = host_data[var].broadcast_like(tele_data["time"])
        
tele_host = host_data
for var in tele_host.variables:
    tele_host[var] = tele_host[var].fillna(value=tele_data[var])

In [ ]:
# Merge the telemetered dataset and host_dataset
#tele_host = tele_data.combine_first(host_data)
#tele_host = host_data

In [ ]:
for var in tele_host.variables:
    if var not in inst_data.variables:
        inst_data[var] = tele_host[var].broadcast_like(inst_data["time"])

for var in inst_data.variables:
    if var not in tele_host.variables:
        tele_host[var] = inst_data[var].broadcast_like(tele_host["time"])
        
data = inst_data
for var in data:
    data[var] = data[var].fillna(value=tele_host[var])

#### Save the combined dataset as a netCDF file

In [ ]:
data.to_netcdf(f"../data/{refdes}_combined.nc", engine="h5netcdf")